In [1]:
# Safety net: are you sure? input 
sure = input("Are you sure you want a new year? (y/n): ")
if sure.lower() not in ["y", "yes"]:
    print("Exiting")
    exit()

In [2]:
# change directory one up
import os
os.chdir('..')

In [3]:
NEW_YEAR = 1

In [4]:
import shutil
import time
if os.path.exists('fantasy/players.csv'):
    shutil.copy('fantasy/players.csv', f'fantasy/backups/players_{time.strftime("%Y%m%d_%H%M%S")}.csv')
if os.path.exists('fantasy/schedule_league_1.csv'):
    shutil.copy('fantasy/schedule_league_1.csv', f'fantasy/backups/schedule_league_1_{time.strftime("%Y%m%d_%H%M%S")}.csv')
if os.path.exists('fantasy/schedule_league_2.csv'):
    shutil.copy('fantasy/schedule_league_2.csv', f'fantasy/backups/schedule_league_2_{time.strftime("%Y%m%d_%H%M%S")}.csv')
if os.path.exists('fantasy/schedule_league_3.csv'):
    shutil.copy('fantasy/schedule_league_3.csv', f'fantasy/backups/schedule_league_3_{time.strftime("%Y%m%d_%H%M%S")}.csv')
if os.path.exists('fantasy/schedule_league_4.csv'):
    shutil.copy('fantasy/schedule_league_4.csv', f'fantasy/backups/schedule_league_4_{time.strftime("%Y%m%d_%H%M%S")}.csv')


In [5]:
# import players.csv as pandas dataframe
import pandas as pd
players = pd.read_csv(f'fantasy/history/year_{NEW_YEAR - 1}/players_year_{NEW_YEAR - 1}.csv')

,Name,Sex,Country,AVG,League
0,Adam Novák,Male,Czech Republic,47.12,1
1,Sofia Martinez,Female,Argentina,45.40,1
2,Hugo Fernández,Male,Spain,44.91,1
3,Owen Jones,Male,Wales,39.59,1
4,Louis Dubois,Male,France,42.44,1


# Average Shift
+ Difference between Average and Season Average
+ Degreation Penalty
+ Advancement Bonus
+ Parity Bonus & Penalty (League 1)

In [6]:
# use function from standings_table.py (same folder)
import numpy as np
from standings_table import get_standings_table_as_dataframe
players['Change of AVG'] = 0

for league in range(1, 5):
    league_standings = get_standings_table_as_dataframe(league=league, stats=False, file_location_players=f'fantasy/history/year_{NEW_YEAR - 1}/players_year_{NEW_YEAR - 1}.csv', folder_location_schedule=f'fantasy/history/year_{NEW_YEAR - 1}')

    # add AVG from players to standings_league_1 (Name matches in both tables)
    league_standings = pd.merge(league_standings, players, on='Name')

    # remove row with name == PLAYER
    league_standings = league_standings[league_standings['Name'] != 'PLAYER']

    # have a ranking of 'Season Average' in the standings table
    league_standings['Season Average Rank'] = league_standings['Season Average'].rank(ascending=False)

    # have a ranking of 'AVG' in the standings table
    league_standings['AVG Rank'] = league_standings['AVG'].rank(ascending=False)

    # calculate difference between Season Average Rank and AVG Rank
    league_standings['AVG Rank Difference'] = league_standings['AVG Rank'] - league_standings['Season Average Rank']

    league_standings['Degration Penalty'] = league_standings['#'].apply(lambda x: -1 if x > 8 else 0)
    league_standings['Advancement Bonus'] = league_standings['#'].apply(lambda x: 1 if league > 1 and x < 3 else 0)

    league_standings['Parity Bonus'] = league_standings['#'].apply(lambda x: 0.5 if league==1 and x > 1 and x < 8 else -1 if league==1 and x == 1 else 0)

    league_standings['Mean Change of AVG'] = league_standings['AVG Rank Difference'] + league_standings['Degration Penalty'] + league_standings['Advancement Bonus'] + league_standings['Parity Bonus']

    # norm standings_league_1['Mean Change of AVG'] to between -1 and 1
    league_standings['Mean Change of AVG'] = league_standings['Mean Change of AVG'] / league_standings['Mean Change of AVG'].max()

    # actual change of avg = Mean Change of AVG with gaussian noise (normal distribution with std=0.5)
    league_standings['Change of AVG'] = league_standings['Mean Change of AVG'] + np.random.normal(0, 0.5, league_standings.shape[0])

    for index, row in league_standings.iterrows():
        players.loc[players['Name'] == row['Name'], 'Change of AVG'] = row['Change of AVG']

players['New AVG'] = round(players['AVG'] + players['Change of AVG'],2)

/tmp/ipykernel_25827/3696136484.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.08238324863442958' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  players.loc[players['Name'] == row['Name'], 'Change of AVG'] = row['Change of AVG']


# Relegation

In [7]:
players['New League'] = 0
for league in range(1, 5):
    league_standings = get_standings_table_as_dataframe(league=league, stats=False, file_location_players=f'fantasy/history/year_{NEW_YEAR - 1}/players_year_{NEW_YEAR - 1}.csv', folder_location_schedule=f'fantasy/history/year_{NEW_YEAR - 1}')
    league_standings['New League'] = league
    # degrade everybody with # smaller than 8
    league_standings['New League'] = league_standings['#'].apply(lambda x: league + 1 if x > 8 else league -1 if x < 3 and league > 1 else league)
    # if New League > 5 -> NaN
    league_standings['New League'] = league_standings['New League'].apply(lambda x: x if x < 5 else np.nan)

    # only keep name and New League
    league_standings = league_standings[['Name', 'New League']]
    
    for index, row in league_standings.iterrows():
        players.loc[players['Name'] == row['Name'], 'New League'] = row['New League']

# Generate 2 New Players

In [8]:
from names import names
from random import choice

def generate_new_player():

    county_probabilities = {
        'England': 20, 'Wales': 12, 'Scotland': 12, 'Ireland': 8, 'Spain': 2, 'France': 3,
        'Portugal': 2, 'Germany': 5, 'Netherlands': 4, 'Belgium': 1, 'Denmark': 2, 'Norway': 1,
        'Sweden': 1, 'Finland': 1, 'Iceland': 1, 'Italy': 1, 'USA': 3, 'Canada': 3, 'Australia': 4, 'New Zealand': 2,
        'China': 2, 'Japan': 2, 'South Korea': 2, 'Brazil': 2, 'Nigeria': 1, 'Colombia': 2, 'Argentina': 1,
        'Peru': 1, 'Chile': 1, 'Cuba': 1, 'Mexico': 2, 'Serbia': 1, 'Croatia': 1, 'Russia': 2, 'Ukraine': 1,
        'Turkey': 1, 'Israel': 1, 'India': 1, 'Austria': 2, 'Lithuania': 1, 'Greece': 1, 'South Africa': 1
    }

    sex_probabilities = {'Male': 9, 'Female': 1}

    # make choice with probabilities
    def weighted_choice(choices):
        total = sum(w for c, w in choices.items())
        r = np.random.uniform(0, total)
        upto = 0
        for c, w in choices.items():
            if upto + w >= r:
                return c
            upto += w
        assert False, "Shouldn't get here"

    country = weighted_choice(county_probabilities)

    sex = weighted_choice(sex_probabilities)

    first_name = choice(names[country][f'first_names_{sex.lower()}'])
    last_name = choice(names[country][f'last_names'])

    # avg = gausian distribution with mean 28 and std 5
    avg = np.random.normal(29, 4)
    # if avg < 25 -> new gaussian distribution with mean 26 and std 1
    avg = np.random.normal(26, 1) if avg < 24 else avg

    return {
        'first_name': first_name,
        'last_name': last_name,
        'country': country,
        'sex': sex,
        'avg': avg
    }

for i in range(2):
    new_player = generate_new_player()
    print(new_player)
    new_player = pd.DataFrame({'Name': [f'{new_player['first_name']} {new_player['last_name']}'], 'Country': [new_player['country']], 'Sex': [new_player['sex']], 'New AVG': [round(new_player['avg'], 2)], 'New League': [4]})
    players = pd.concat([players, new_player], ignore_index=True)


{'first_name': 'Dennis', 'last_name': 'Jones', 'country': 'USA', 'sex': 'Male', 'avg': 24.43040904144445}
{'first_name': 'Owen', 'last_name': 'Carter', 'country': 'England', 'sex': 'Male', 'avg': 29.102095630725575}


# Report

In [9]:
avg_league = players.groupby('New League')['New AVG'].mean()
old_avg_league = players.groupby('League')['AVG'].mean()
avg_league = pd.merge(avg_league, old_avg_league, left_index=True, right_index=True)
avg_league['Change in AVG'] = avg_league['New AVG'] - avg_league['AVG']
avg_league = avg_league.rename(columns={'AVG': 'Old AVG'})
avg_league = avg_league.round(2)
avg_league = avg_league[['Old AVG', 'New AVG', 'Change in AVG']]
avg_league

,Old AVG,New AVG,Change in AVG
New League,,,
1.0,43.53,43.41,-0.12
2.0,37.86,38.24,0.38
3.0,33.58,33.52,-0.06
4.0,29.11,29.64,0.53


# Export

In [10]:
players = players.sort_values(by=['New League', 'League', 'New AVG'], ascending=[True, True, False])
# drop League
players.drop(columns=['League', 'AVG', 'Change of AVG'], inplace=True)

# rename New League to League and New AVG to AVG
players.rename(columns={'New League': 'League', 'New AVG': 'AVG'}, inplace=True)

# datatype of League to int (or nan)
players['League'] = players['League'].astype('Int64')

# save new players.csv
players.to_csv(f'fantasy/players.csv', index=False)

players

,Name,Sex,Country,AVG,League
0,Adam Novák,Male,Czech Republic,47.20,1
2,Hugo Fernández,Male,Spain,45.14,1
1,Sofia Martinez,Female,Argentina,44.87,1
8,Lukas Kazlauskas,Male,Lithuania,44.83,1
9,Charlie Wilson,Male,England,44.79,1
5,Gareth Williams,Male,Wales,42.91,1
4,Louis Dubois,Male,France,42.77,1
7,Ryan Fraser,Male,Scotland,39.41,1
18,Lucas de Jong,Male,Netherlands,41.43,1
12,Maximilian Sägemann,Male,Germany,40.79,1


# Schedule Creation

In [11]:
for league in range(1,5):
    players_in_league = players[players['League'] == league]

    games = pd.DataFrame(columns=['Day', 'Home Player', 'Away Player', 'Result', 'Game Log'])

    num_days = (len(players_in_league) - 1) * 2 
    half_size = len(players_in_league) // 2

    # put names column of players_in_league into list
    players_list = players_in_league['Name'].tolist()

    for day in range(num_days):
        for i in range(half_size):
            if players_list[i] is not None and players_list[-i-1] is not None:
                if day < len(players_list) - 1:  # First half of the schedule
                    games.loc[len(games)] = [day+1, players_list[i], players_list[-i-1], None, None]
                else:  # Second half of the schedule
                    games.loc[len(games)] = [day+1, players_list[-i-1], players_list[i], None, None]

        # Rotate the list of players_list except the first one
        players_list = players_list[:1] + players_list[-1:] + players_list[1:-1]

    # give me a list of numbers 10 to 18 but shuffled
    days = list(range(10, 19))
    np.random.shuffle(days)

    # make dict with day (10 to 18) and the shuffled days
    day_dict = {i + 10: days[i] for i in range(len(days))}

    # replace the day with the shuffled day
    games['Day'] = games['Day'].apply(lambda x: day_dict[x] if x > 9 else x)

    # sort by Day
    games = games.sort_values(by='Day')

    games.to_csv(f'fantasy/schedule_league_{league}.csv', index=False)